In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score#准确率
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler,LabelEncoder#标签编码

In [4]:
data=pd.read_csv('/Users/liuriyilang/Desktop/课件/机器学习/机器学习和算法/01 KNN机器学习入门/代码/adults.txt')
display(data.head(),data.shape)

,age,workclass,final_weight,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


(32561, 15)

# 数据预处理

In [5]:
# 出利空数据
data.isna().sum()

age               0
workclass         0
final_weight      0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
salary            0
dtype: int64

In [6]:
data['workclass'].unique()
#?就是空数据

array(['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov',
       'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked'],
      dtype=object)

In [10]:
data=data.replace("?",np.nan).dropna().reset_index(drop=True)
#replace是替换

In [11]:
data.shape

(30162, 15)

In [12]:
data.head()

,age,workclass,final_weight,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [14]:
#异常值，算法建模不是好数据
data['capital_gain'].sort_values(ascending=False).unique()

array([99999, 41310, 34095, 27828, 25236, 25124, 22040, 20051, 18481,
       15831, 15024, 15020, 14344, 14084, 13550, 11678, 10605, 10566,
       10520,  9562,  9386,  8614,  7978,  7896,  7688,  7443,  7430,
        7298,  6849,  6767,  6723,  6514,  6497,  6418,  6360,  6097,
        5721,  5556,  5455,  5178,  5060,  5013,  4934,  4931,  4865,
        4787,  4687,  4650,  4508,  4416,  4386,  4101,  4064,  3942,
        3908,  3887,  3818,  3781,  3674,  3471,  3464,  3456,  3432,
        3418,  3411,  3325,  3273,  3137,  3103,  2993,  2977,  2964,
        2961,  2936,  2907,  2885,  2829,  2653,  2635,  2597,  2580,
        2538,  2463,  2414,  2407,  2387,  2354,  2346,  2329,  2290,
        2228,  2202,  2176,  2174,  2105,  2062,  2050,  2036,  2009,
        1848,  1831,  1797,  1639,  1506,  1471,  1455,  1424,  1409,
        1173,  1151,  1086,  1055,   991,   914,   594,   401,   114,
           0])

In [15]:
data['capital_loss'].sort_values(ascending=False).unique()

array([4356, 3900, 3770, 3683, 3004, 2824, 2754, 2603, 2559, 2547, 2472,
       2467, 2457, 2444, 2415, 2392, 2377, 2352, 2339, 2282, 2267, 2258,
       2246, 2238, 2231, 2206, 2205, 2201, 2179, 2174, 2149, 2129, 2080,
       2057, 2051, 2042, 2002, 2001, 1980, 1977, 1974, 1944, 1902, 1887,
       1876, 1848, 1844, 1825, 1816, 1762, 1755, 1741, 1740, 1735, 1726,
       1721, 1719, 1672, 1669, 1668, 1651, 1648, 1628, 1617, 1602, 1594,
       1590, 1579, 1573, 1564, 1539, 1504, 1485, 1411, 1408, 1380, 1340,
       1258, 1138, 1092,  974,  880,  810,  653,  625,  419,  323,  213,
        155,    0])

In [16]:
cond=(data['capital_gain']!=99999) & (data['capital_loss']!=4356)
data=data[cond]
data.shape

(30013, 15)

In [17]:
#字符串-》数值型数据
le=LabelEncoder()#为字符串排序
le.fit(['paris','paris','tokyo','amsterdam'])
le.transform(['paris','paris','tokyo','amsterdam'])

array([1, 1, 2, 0])

In [21]:
columns=['workclass','education','marital_status','occupation','relationship','race','sex','native_country']
le=LabelEncoder()
for col in columns:
    data[col]=le.fit_transform(data[col])
data.head()

,age,workclass,final_weight,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
0,0.301370,5,0.043338,9,0.800000,4,0,1,4,1,0.052626,0.0,0.397959,38,<=50K
1,0.452055,4,0.047277,9,0.800000,2,3,0,4,1,0.000000,0.0,0.122449,38,<=50K
2,0.287671,2,0.137244,11,0.533333,0,5,1,4,1,0.000000,0.0,0.397959,38,<=50K
3,0.493151,2,0.150212,1,0.400000,2,5,0,2,1,0.000000,0.0,0.397959,38,<=50K
4,0.150685,2,0.220703,9,0.800000,2,9,5,2,0,0.000000,0.0,0.397959,4,<=50K


In [23]:
#归一化min-max
scaler=MinMaxScaler()
col_norm=['age','final_weight','workclass','education','education_num','marital_status',
          'occupation','relationship','race','capital_gain','capital_loss','hours_per_week',
          'native_country']
data[col_norm]=scaler.fit_transform(data[col_norm])
data.head()

,age,workclass,final_weight,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
0,0.301370,0.833333,0.043338,0.600000,0.800000,0.666667,0.000000,0.2,1.0,1,0.052626,0.0,0.397959,0.95,<=50K
1,0.452055,0.666667,0.047277,0.600000,0.800000,0.333333,0.230769,0.0,1.0,1,0.000000,0.0,0.122449,0.95,<=50K
2,0.287671,0.333333,0.137244,0.733333,0.533333,0.000000,0.384615,0.2,1.0,1,0.000000,0.0,0.397959,0.95,<=50K
3,0.493151,0.333333,0.150212,0.066667,0.400000,0.333333,0.384615,0.0,0.5,1,0.000000,0.0,0.397959,0.95,<=50K
4,0.150685,0.333333,0.220703,0.600000,0.800000,0.333333,0.692308,1.0,0.5,0,0.000000,0.0,0.397959,0.10,<=50K


# 数据拆分

In [24]:
y=data['salary']
X=data.drop(labels='salary',axis=1)
X.shape

(30013, 14)

In [25]:
X_trin,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1024)

In [26]:
knn=KNeighborsClassifier()
knn.fit(X_trin,y_train)
y_pre=knn.predict(X_test)
print('true value',y_test[:20].values)
print('predict values',y_pre[:20])

true value ['<=50K' '<=50K' '<=50K' '<=50K' '<=50K' '<=50K' '<=50K' '<=50K' '>50K'
 '<=50K' '<=50K' '<=50K' '<=50K' '>50K' '<=50K' '<=50K' '>50K' '<=50K'
 '>50K' '<=50K']
predict values ['<=50K' '<=50K' '<=50K' '<=50K' '<=50K' '<=50K' '<=50K' '<=50K' '<=50K'
 '<=50K' '<=50K' '<=50K' '<=50K' '<=50K' '>50K' '<=50K' '>50K' '<=50K'
 '>50K' '<=50K']


/Users/liuriyilang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [27]:
accuracy_score(y_test,y_pre)

0.8204231217724471

# 模型调优

In [28]:
%%time
knn=KNeighborsClassifier()

params=dict( n_neighbors=[3,5,9,15,23,31,39],weights=['uniform','distance'],p=[1,2])

gCV=GridSearchCV(knn,params,cv=5)
gCV.fit(X_trin,y_train)

/Users/liuriyilang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/liuriyilang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will

CPU times: user 1min 26s, sys: 803 ms, total: 1min 27s
Wall time: 1min 27s


GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 9, 15, 23, 31, 39], 'p': [1, 2],
                         'weights': ['uniform', 'distance']})

In [29]:
gCV.best_params_

{'n_neighbors': 23, 'p': 1, 'weights': 'uniform'}

In [30]:
knn_best=gCV.best_estimator_
knn_best.score(X_test,y_test)

/Users/liuriyilang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.834416125270698